##CEPARCO-S11
*CUDA Implementation of ROBINSON MASK*
```
PROPONENTS:

    ALONZO, Jose Anton

    AVELINO, Joris Gabriel

    CRUZ, Airon John

    HERNANDEZ, Pierre Vincent
```



## Robinson Compass Mask CUDA

In [ ]:
%%writefile robinsonComp_CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

// Headers for image input and output
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#define CUDA_CHECK(call)                                            \
do {                                                                \
    cudaError_t cudaStatus = call;                                  \
    if (cudaStatus != cudaSuccess) {                                \
        printf("CUDA Error at line %d: %s\n", __LINE__, cudaGetErrorString(cudaStatus)); \
        exit(1);                                                    \
    }                                                               \
} while (0)


#define MAX_CHAR 32
#define GRAY 1
#define RGB 3
#define MASK_SIZE 3


// ===================================================================================================

// CUDA kernel to apply the Robinson Compass Mask
__global__ void robCompMaskKernel(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    int h = blockIdx.y * blockDim.y + threadIdx.y; //row of the pixel
    int w = blockIdx.x * blockDim.x + threadIdx.x;  // col of the pixel

    if (h < height && w < width) {
        int sum = 0;
        for (int r = -1; r <= 1; r++) {
            for (int c = -1; c <= 1; c++) {
                int idx_h = min(max(h + r, 0), height - 1);
                int idx_w = min(max(w + c, 0), width - 1);
                sum += img_in[idx_h * width + idx_w] * mask[(r + 1) * 3 + (c + 1)];
            }
        }
        img_out[h * width + w] = (unsigned char) abs(sum);
    }
}

// Function to apply the Robinson Compass Mask using CUDA
void robCompMaskCUDA(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    unsigned char *d_img_in, *d_img_out;
    int *d_mask;

    // Allocate device memory for input image, output image, and mask
    CUDA_CHECK(cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_mask, 9 * sizeof(int)));

    // Copy data from host to device
    CUDA_CHECK(cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mask, mask, 9 * sizeof(int), cudaMemcpyHostToDevice));
    // Define block size and grid size
    dim3 blockSize(32, 32);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    robCompMaskKernel<<<gridSize, blockSize>>>(d_img_out, d_img_in, width, height, d_mask);
    cudaDeviceSynchronize();

    // Copy result back to host
    CUDA_CHECK(cudaMemcpy(img_out, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Free device memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}

void generateOutputFilename(char* filename, char* output_filename, const char* direction) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != '.'; i++) {
        output_filename[i + 10] = filename[i];
    }
    output_filename[i + 10] = '_';
    i++;
    // copy the direction to the output filename
    int j;
    for (j = 0; direction[j] != 0; j++) {
        output_filename[i + 10 + j] = direction[j];
    }
    // add the filetype to the output filename
    output_filename[i + 10 + j] = '.';
    output_filename[i + 10 + j + 1] = 'j';
    output_filename[i + 10 + j + 2] = 'p';
    output_filename[i + 10 + j + 3] = 'g';
    output_filename[i + 10 + j + 4] = 0;
}

void generateInputFilename(char* filename, char* input_filename) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != 0; i++) {
        input_filename[i + 9] = filename[i];
    }
}

int isFileTypeSupported(char* filename) {
    // Declare supported filetypes
    const char* supported_filetypes[] = {
        "jpg", "jpeg", "png", "bmp"
    };
    // Move to the period before the filetype
    int start = 0;
    for (int i = 0; i < MAX_CHAR && filename[start] != '.'; i++, start++);
    start++; // move to the first character of the filetype

    // Check if the filetype is supported
    for (int i = start, j = 0; j < 4 && filename[i] != 0; i++, j++) {
        // Debugging purposes: print the filename extension and the supported filetypes
        /* printf("%c - %c - %c - %c - %c\n",
            filename[i],
            supported_filetypes[0][j % 3],
            supported_filetypes[1][j % 4],
            supported_filetypes[2][j % 3],
            supported_filetypes[3][j % 3]); */

        // Check if the character is not equal to any of the supported file types
        if (filename[i] != supported_filetypes[0][j % 3]
            && filename[i] != supported_filetypes[1][j % 4]
            && filename[i] != supported_filetypes[2][j % 3]
            && filename[i] != supported_filetypes[3][j % 3]) {
            return 0; // return 0 if the filetype is not supported
        }
    }
    return 1; // return 1 if the filetype is supported
}

int main() {
    const int NUM_DIRECTIONS = 8;

    // Declare Robinson's Compass mask
      const int ROBINSON_COMPASS_MASK[8][3][3] = {
        {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}}, // N
        {{0, 1, 2}, {-1, 0, 1}, {-2, -1, 0}}, // NW
        {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}}, // W
        {{2, 1, 0}, {1, 0, -1}, {0, -1, -2}}, // SW
        {{1, 0, -1}, {2, 0, -2}, {1, 0, -1}}, // S
        {{0, -1, -2}, {1, 0, -1}, {2, 1, 0}}, // SE
        {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}}, // E
        {{-2, -1, 0}, {-1, 0, 1}, {0, 1, 2}}  // NE
    };


    int width, height, channels;
    char filename[MAX_CHAR] = { 0 };
    char input_filename[MAX_CHAR + 16] = "./inputs/\0";
    char output_filename[MAX_CHAR + 16] = "./outputs/\0";
    unsigned char *img_in;

    // Ask for the image filename
    printf("Enter the image filename: ");
    scanf("%[^\n]%*c", filename);
    // Debugging purposes: print filename
    printf("Filename: %s\n", filename); // expected to print only one newline char

    // Check if filetype is supported for this program
    if (!isFileTypeSupported(filename)) {
        printf("File type not supported.\n");
        printf("Image should either be in JPG, PNG or BMP format.\n");
        return 1; // exit the program with 1 error
    }

    // Generate the input filename
    generateInputFilename(filename, input_filename);
    // Debugging purposes: print the input filename
    printf("Input filename: %s\n", input_filename);

    // Read the image
    img_in = stbi_load(input_filename, &width, &height, &channels, GRAY); // 4th param: GRAY or RGB
    // check image if it exists
    if (img_in == NULL) {
        printf("Error in loading the image.\n");
        printf("Please check if the image exists.\n");
        return 1; // exit the program with 1 error
    }
    // Debugging purposes: print image width, height, and channels
    printf("Image width: %d\n", width);
    printf("Image height: %d\n", height);
    printf("Number of channels: %d\n", channels);

    // Debugging purposes: to check if the image is in grayscale
    if (!stbi_write_jpg("./grayscale/input.jpg", width, height, GRAY, img_in, 100)) {
        printf("Error in saving the output image.\n");
        return 1; // exit the program with 1 error
    }

    // Allocate memory for the output images
    unsigned char *img_out[NUM_DIRECTIONS];
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        img_out[i] = (unsigned char *)malloc(width * height * sizeof(unsigned char));
        if (img_out[i] == NULL) {
            printf("Error in allocating memory for the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Apply the Robinson's Compass mask for each direction using CUDA
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        robCompMaskCUDA(img_out[i], img_in, width, height, &ROBINSON_COMPASS_MASK[i][0][0]);
        // Generate output filename
        char direction[MAX_CHAR];
        snprintf(direction, MAX_CHAR, "Direction_%d", i);
        generateOutputFilename(filename, output_filename, direction);
        // Debugging purposes: print output filename
        printf("Output filename: %s\n", output_filename);
        // Save output image
        if (!stbi_write_jpg(output_filename, width, height, GRAY, img_out[i], 100)) {
            printf("Error in saving the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Free memory for the output images
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        free(img_out[i]);
    }

    // Free memory for the input image
    stbi_image_free(img_in);

    return 0;
}

Writing robinsonComp_CUDA.cu


In [ ]:
%%shell
nvcc robinsonComp_CUDA.cu -o robinsonComp_CUDA

stb_image.h(4262): warning #550-D: variable "old_limit" was set but never used

stb_image.h(5184): warning #550-D: variable "idata_limit_old" was set but never used

stb_image.h(6971): warning #550-D: variable "out_size" was set but never used

stb_image.h(6972): warning #550-D: variable "delays_size" was set but never used



In [ ]:
%%shell
nvprof ./robinsonComp_CUDA

Enter the image filename: cat_small.jpg
Filename: cat_small.jpg
Input filename: ./inputs/cat_small.jpg
Image width: 224
Image height: 225
Number of channels: 3
==2841== NVPROF is profiling process 2841, command: ./robinsonComp_CUDA
Output filename: ./outputs/cat_small_Direction_0.jpg
Output filename: ./outputs/cat_small_Direction_1.jpg
Output filename: ./outputs/cat_small_Direction_2.jpg
Output filename: ./outputs/cat_small_Direction_3.jpg
Output filename: ./outputs/cat_small_Direction_4.jpg
Output filename: ./outputs/cat_small_Direction_5.jpg
Output filename: ./outputs/cat_small_Direction_6.jpg
Output filename: ./outputs/cat_small_Direction_7.jpg
==2841== Profiling application: ./robinsonComp_CUDA
==2841== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.88%  83.966us         8  10.495us  10.304us  10.848us  robCompMaskKernel(unsigned char*, unsigned char*, int, int, int const *)
                   36.66%  73.501

## Robinson Compass Mask CUDA (with Grid-stride loop)

In [ ]:
%%writefile robinsonComp_gsCUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

// Headers for image input and output
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#define CUDA_CHECK(call)                                            \
do {                                                                \
    cudaError_t cudaStatus = call;                                  \
    if (cudaStatus != cudaSuccess) {                                \
        printf("CUDA Error at line %d: %s\n", __LINE__, cudaGetErrorString(cudaStatus)); \
        exit(1);                                                    \
    }                                                               \
} while (0)


#define MAX_CHAR 32
#define GRAY 1
#define RGB 3
#define MASK_SIZE 3


// ===================================================================================================

// CUDA kernel to apply the Robinson Compass Mask
__global__ void robCompMaskKernel(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < width * height; i += stride) {
        int h = i / width;
        int w = i % width;

        if (h < height && w < width) {
            int sum = 0;
            for (int r = -1; r <= 1; r++) {
                for (int c = -1; c <= 1; c++) {
                    int idx_h = min(max(h + r, 0), height - 1);
                    int idx_w = min(max(w + c, 0), width - 1);
                    sum += img_in[idx_h * width + idx_w] * mask[(r + 1) * 3 + (c + 1)];
                }
            }
            img_out[h * width + w] = (unsigned char)abs(sum);
        }
    }
}


// Function to apply the Robinson Compass Mask using CUDA
void robCompMaskCUDA(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    unsigned char *d_img_in, *d_img_out;
    int *d_mask;

    // Allocate device memory for input image, output image, and mask
    CUDA_CHECK(cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_mask, 9 * sizeof(int)));

    // Copy data from host to device
    CUDA_CHECK(cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mask, mask, 9 * sizeof(int), cudaMemcpyHostToDevice));
    // Define block size and grid size
        //start here
    int numThreads = 1024;
    int numBlocks = ((width*height)+numThreads-1) / numThreads;
    //dim3 blockSize(32, 32); //same as 1024 threads per block
    //dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    robCompMaskKernel<<<numBlocks,  numThreads>>>(d_img_out, d_img_in, width, height, d_mask);

    // Copy result back to host
    CUDA_CHECK(cudaMemcpy(img_out, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Free device memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}

void generateOutputFilename(char* filename, char* output_filename, const char* direction) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != '.'; i++) {
        output_filename[i + 10] = filename[i];
    }
    output_filename[i + 10] = '_';
    i++;
    // copy the direction to the output filename
    int j;
    for (j = 0; direction[j] != 0; j++) {
        output_filename[i + 10 + j] = direction[j];
    }
    // add the filetype to the output filename
    output_filename[i + 10 + j] = '.';
    output_filename[i + 10 + j + 1] = 'j';
    output_filename[i + 10 + j + 2] = 'p';
    output_filename[i + 10 + j + 3] = 'g';
    output_filename[i + 10 + j + 4] = 0;
}

void generateInputFilename(char* filename, char* input_filename) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != 0; i++) {
        input_filename[i + 9] = filename[i];
    }
}

int isFileTypeSupported(char* filename) {
    // Declare supported filetypes
    const char* supported_filetypes[] = {
        "jpg", "jpeg", "png", "bmp"
    };
    // Move to the period before the filetype
    int start = 0;
    for (int i = 0; i < MAX_CHAR && filename[start] != '.'; i++, start++);
    start++; // move to the first character of the filetype

    // Check if the filetype is supported
    for (int i = start, j = 0; j < 4 && filename[i] != 0; i++, j++) {
        // Debugging purposes: print the filename extension and the supported filetypes
        /* printf("%c - %c - %c - %c - %c\n",
            filename[i],
            supported_filetypes[0][j % 3],
            supported_filetypes[1][j % 4],
            supported_filetypes[2][j % 3],
            supported_filetypes[3][j % 3]); */

        // Check if the character is not equal to any of the supported file types
        if (filename[i] != supported_filetypes[0][j % 3]
            && filename[i] != supported_filetypes[1][j % 4]
            && filename[i] != supported_filetypes[2][j % 3]
            && filename[i] != supported_filetypes[3][j % 3]) {
            return 0; // return 0 if the filetype is not supported
        }
    }
    return 1; // return 1 if the filetype is supported
}

int main() {
    const int NUM_DIRECTIONS = 8;

    // Declare Robinson's Compass mask
      const int ROBINSON_COMPASS_MASK[8][3][3] = {
        {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}}, // N
        {{0, 1, 2}, {-1, 0, 1}, {-2, -1, 0}}, // NW
        {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}}, // W
        {{2, 1, 0}, {1, 0, -1}, {0, -1, -2}}, // SW
        {{1, 0, -1}, {2, 0, -2}, {1, 0, -1}}, // S
        {{0, -1, -2}, {1, 0, -1}, {2, 1, 0}}, // SE
        {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}}, // E
        {{-2, -1, 0}, {-1, 0, 1}, {0, 1, 2}}  // NE
    };


    int width, height, channels;
    char filename[MAX_CHAR] = { 0 };
    char input_filename[MAX_CHAR + 16] = "./inputs/\0";
    char output_filename[MAX_CHAR + 16] = "./outputs/\0";
    unsigned char *img_in;

    // Ask for the image filename
    printf("Enter the image filename: ");
    scanf("%[^\n]%*c", filename);
    // Debugging purposes: print filename
    printf("Filename: %s\n", filename); // expected to print only one newline char

    // Check if filetype is supported for this program
    if (!isFileTypeSupported(filename)) {
        printf("File type not supported.\n");
        printf("Image should either be in JPG, PNG or BMP format.\n");
        return 1; // exit the program with 1 error
    }

    // Generate the input filename
    generateInputFilename(filename, input_filename);
    // Debugging purposes: print the input filename
    printf("Input filename: %s\n", input_filename);

    // Read the image
    img_in = stbi_load(input_filename, &width, &height, &channels, GRAY); // 4th param: GRAY or RGB
    // check image if it exists
    if (img_in == NULL) {
        printf("Error in loading the image.\n");
        printf("Please check if the image exists.\n");
        return 1; // exit the program with 1 error
    }
    // Debugging purposes: print image width, height, and channels
    printf("Image width: %d\n", width);
    printf("Image height: %d\n", height);
    printf("Number of channels: %d\n", channels);

    // Debugging purposes: to check if the image is in grayscale
    if (!stbi_write_jpg("./grayscale/input.jpg", width, height, GRAY, img_in, 100)) {
        printf("Error in saving the output image.\n");
        return 1; // exit the program with 1 error
    }

    // Allocate memory for the output images
    unsigned char *img_out[NUM_DIRECTIONS];
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        img_out[i] = (unsigned char *)malloc(width * height * sizeof(unsigned char));
        if (img_out[i] == NULL) {
            printf("Error in allocating memory for the output image.\n");
            return 1; // exit the program with 1 error
        }
    }
    // Apply the Robinson's Compass mask for each direction using CUDA
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        robCompMaskCUDA(img_out[i], img_in, width, height, &ROBINSON_COMPASS_MASK[i][0][0]);
        // Generate output filename
        char direction[MAX_CHAR];
        snprintf(direction, MAX_CHAR, "Direction_%d", i);
        generateOutputFilename(filename, output_filename, direction);
        // Debugging purposes: print output filename
        printf("Output filename: %s\n", output_filename);
        // Save output image
        if (!stbi_write_jpg(output_filename, width, height, GRAY, img_out[i], 100)) {
            printf("Error in saving the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Free memory for the output images
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        free(img_out[i]);
    }

    // Free memory for the input image
    stbi_image_free(img_in);

    return 0;
}

In [ ]:
%%shell
nvcc robinsonComp_gsCUDA.cu -o robinsonComp_gsCUDA

In [ ]:
%%shell
nvprof ./robinsonComp_gsCUDA.cu

## Robinson Compass Mask CUDA with Gaussian Blur

In [ ]:
%%writefile robinsonCompGBlur_CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

// Headers for image input and output
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#define CUDA_CHECK(call)                                            \
do {                                                                \
    cudaError_t cudaStatus = call;                                  \
    if (cudaStatus != cudaSuccess) {                                \
        printf("CUDA Error at line %d: %s\n", __LINE__, cudaGetErrorString(cudaStatus)); \
        exit(1);                                                    \
    }                                                               \
} while (0)


#define MAX_CHAR 32
#define GRAY 1
#define RGB 3
#define MASK_SIZE 3
#define GAUSSIAN_MASK_SIZE 9  // positive odd number >= 3
#define GAUSSIAN_SIGMA 1.0    // postive number >= 0.3

// For the PI value in C
#ifndef M_PI
#   define M_PI 3.14159265358979323846
#endif



// =================================RCM KERNEL====================================================

// CUDA kernel to apply the Robinson Compass Mask
__global__ void robCompMaskKernel(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    int h = blockIdx.y * blockDim.y + threadIdx.y;
    int w = blockIdx.x * blockDim.x + threadIdx.x;

    if (h < height && w < width) {
        int sum = 0;
        for (int r = -1; r <= 1; r++) {
            for (int c = -1; c <= 1; c++) {
                int idx_h = min(max(h + r, 0), height - 1);
                int idx_w = min(max(w + c, 0), width - 1);
                sum += img_in[idx_h * width + idx_w] * mask[(r + 1) * 3 + (c + 1)];
            }
        }
        img_out[h * width + w] = (unsigned char) abs(sum);
    }
}

// Function to apply the Robinson Compass Mask using CUDA
void robCompMaskCUDA(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    unsigned char *d_img_in, *d_img_out;
    int *d_mask;

    // Allocate device memory for input image, output image, and mask
    CUDA_CHECK(cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_mask, 9 * sizeof(int)));

    // Copy data from host to device
    CUDA_CHECK(cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mask, mask, 9 * sizeof(int), cudaMemcpyHostToDevice));
    // Define block size and grid size
    dim3 blockSize(32, 32);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    robCompMaskKernel<<<gridSize, blockSize>>>(d_img_out, d_img_in, width, height, d_mask);

    // Copy result back to host
    CUDA_CHECK(cudaMemcpy(img_out, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Free device memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}

// =================================GAUSSIAN BLUR (FOR SMOOTHENING)====================================================
void generateGaussianBlurMask(double mask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE]) {
    // Declare the Gaussian mask
    double sum = 0;
    int max_index = GAUSSIAN_MASK_SIZE / 2;
    for (int r = -GAUSSIAN_MASK_SIZE/2; r <= max_index; r++) {
        for (int c = -GAUSSIAN_MASK_SIZE/2; c <= max_index; c++) {
            mask[r + max_index][c + max_index] = exp(-((double)(r * r + c * c)) / (double)(2 * GAUSSIAN_SIGMA * GAUSSIAN_SIGMA)) / (double)(2 * M_PI * GAUSSIAN_SIGMA * GAUSSIAN_SIGMA);
            sum += mask[r + max_index][c + max_index];
        }
    }

    // Normalize the Gaussian mask
    for (int r = 0; r < GAUSSIAN_MASK_SIZE; r++) {
        for (int c = 0; c < GAUSSIAN_MASK_SIZE; c++) {
            mask[r][c] /= sum;
        }
    }
}

__global__ void gaussianBlurMaskKernel(unsigned char *img_in, unsigned char *img_out, int width, int height, double *mask) {
    const int maxIndex = GAUSSIAN_MASK_SIZE / 2;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < width) {
        int sum = 0;
        int rowStart, colStart, maskRowIndex, maskColIndex, maxRow, maxCol;

        // Reset mask indices
        rowStart = -maxIndex;
        colStart = -maxIndex;
        maskRowIndex = 0;
        maskColIndex = 0;
        maxRow = GAUSSIAN_MASK_SIZE;
        maxCol = GAUSSIAN_MASK_SIZE;

        // Center mask to the current pixel
        if (row == 0) {
            rowStart = 0;
            maskRowIndex = maxIndex;
            maxRow = GAUSSIAN_MASK_SIZE;
        }
        if (row > 0 && row < maxIndex) {
            rowStart = -row;
            maskRowIndex = row;
            maxRow = GAUSSIAN_MASK_SIZE;
        }
        if (row >= height - maxIndex && row < height - 1) {
            rowStart = -maxIndex;
            maskRowIndex = 0;
            maxRow = GAUSSIAN_MASK_SIZE - (height - row - 1);
        }
        if (row == height - 1) {
            rowStart = -maxIndex;
            maskRowIndex = 0;
            maxRow = maxIndex + 1;
        }
        if (col == 0) {
            colStart = 0;
            maskColIndex = maxIndex;
            maxCol = GAUSSIAN_MASK_SIZE;
        }
        if (col > 0 && col < maxIndex) {
            colStart = -col;
            maskColIndex = col;
            maxCol = GAUSSIAN_MASK_SIZE;
        }
        if (col >= width - maxIndex && col < width - 1) {
            colStart = -maxIndex;
            maskColIndex = 0;
            maxCol = GAUSSIAN_MASK_SIZE - (width - col - 1);
        }
        if (col == width - 1) {
            colStart = -maxIndex;
            maskColIndex = 0;
            maxCol = maxIndex + 1;
        }

        // Apply the mask
        for (int r = rowStart, mr = maskRowIndex; mr < maxRow; r++, mr++) {
            for (int c = colStart, mc = maskColIndex; mc < maxCol; c++, mc++) {
                sum += img_in[(row + r) * width + (col + c)] * mask[mr * GAUSSIAN_MASK_SIZE + mc];
            }
        }

        // Set the output pixel to the sum
        img_out[row * width + col] = (unsigned char)abs(sum);
    }
}

void gaussianBlurMask(unsigned char *img_in, int width, int height, double mask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE]) {
    const int TILE_SIZE = 16;

    unsigned char *d_img_in, *d_img_out;
    double *d_mask;

    // Allocate memory on the GPU
    CUDA_CHECK(cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_mask, GAUSSIAN_MASK_SIZE * GAUSSIAN_MASK_SIZE * sizeof(double)));

    // Copy data from host to device
    CUDA_CHECK(cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mask, mask, GAUSSIAN_MASK_SIZE * GAUSSIAN_MASK_SIZE * sizeof(double), cudaMemcpyHostToDevice));

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 numBlocks((width + TILE_SIZE - 1) / TILE_SIZE, (height + TILE_SIZE - 1) / TILE_SIZE);

    gaussianBlurMaskKernel<<<numBlocks, threadsPerBlock>>>(d_img_in, d_img_out, width, height, d_mask);
    cudaDeviceSynchronize();

    // Copy the result back to the host
    CUDA_CHECK(cudaMemcpy(img_in, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Free GPU memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}

// =================================INPUT AND OUTPUT FILE VALIDATOR & GENERATOR=================================

void generateOutputFilename(char* filename, char* output_filename, const char* direction) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != '.'; i++) {
        output_filename[i + 10] = filename[i];
    }
    output_filename[i + 10] = '_';
    i++;
    // copy the direction to the output filename
    int j;
    for (j = 0; direction[j] != 0; j++) {
        output_filename[i + 10 + j] = direction[j];
    }
    // add the filetype to the output filename
    output_filename[i + 10 + j] = '.';
    output_filename[i + 10 + j + 1] = 'j';
    output_filename[i + 10 + j + 2] = 'p';
    output_filename[i + 10 + j + 3] = 'g';
    output_filename[i + 10 + j + 4] = 0;
}

void generateInputFilename(char* filename, char* input_filename) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != 0; i++) {
        input_filename[i + 9] = filename[i];
    }
}

int isFileTypeSupported(char* filename) {
    // Declare supported filetypes
    const char* supported_filetypes[] = {
        "jpg", "jpeg", "png", "bmp"
    };
    // Move to the period before the filetype
    int start = 0;
    for (int i = 0; i < MAX_CHAR && filename[start] != '.'; i++, start++);
    start++; // move to the first character of the filetype

    // Check if the filetype is supported
    for (int i = start, j = 0; j < 4 && filename[i] != 0; i++, j++) {
        // Debugging purposes: print the filename extension and the supported filetypes
        /* printf("%c - %c - %c - %c - %c\n",
            filename[i],
            supported_filetypes[0][j % 3],
            supported_filetypes[1][j % 4],
            supported_filetypes[2][j % 3],
            supported_filetypes[3][j % 3]); */

        // Check if the character is not equal to any of the supported file types
        if (filename[i] != supported_filetypes[0][j % 3]
            && filename[i] != supported_filetypes[1][j % 4]
            && filename[i] != supported_filetypes[2][j % 3]
            && filename[i] != supported_filetypes[3][j % 3]) {
            return 0; // return 0 if the filetype is not supported
        }
    }
    return 1; // return 1 if the filetype is supported
}

int main() {
    const int NUM_DIRECTIONS = 8;

    // Declare Robinson's Compass mask
      const int ROBINSON_COMPASS_MASK[8][3][3] = {
        {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}}, // N
        {{0, 1, 2}, {-1, 0, 1}, {-2, -1, 0}}, // NW
        {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}}, // W
        {{2, 1, 0}, {1, 0, -1}, {0, -1, -2}}, // SW
        {{1, 0, -1}, {2, 0, -2}, {1, 0, -1}}, // S
        {{0, -1, -2}, {1, 0, -1}, {2, 1, 0}}, // SE
        {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}}, // E
        {{-2, -1, 0}, {-1, 0, 1}, {0, 1, 2}}  // NE
    };

    // Declare Gaussian Blur Mask initialized to zero
    double gaussianMask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE] = { { 0 } };

    int width, height, channels;
    char filename[MAX_CHAR] = { 0 };
    char input_filename[MAX_CHAR + 16] = "./inputs/\0";
    char output_filename[MAX_CHAR + 16] = "./outputs/\0";
    unsigned char *img_in;

    // Ask for the image filename
    printf("Enter the image filename: ");
    scanf("%[^\n]%*c", filename);
    // Debugging purposes: print filename
    printf("Filename: %s\n", filename); // expected to print only one newline char

    // Check if filetype is supported for this program
    if (!isFileTypeSupported(filename)) {
        printf("File type not supported.\n");
        printf("Image should either be in JPG, PNG or BMP format.\n");
        return 1; // exit the program with 1 error
    }

    // Generate the input filename
    generateInputFilename(filename, input_filename);
    // Debugging purposes: print the input filename
    printf("Input filename: %s\n", input_filename);

    // Read the image
    img_in = stbi_load(input_filename, &width, &height, &channels, GRAY); // 4th param: GRAY or RGB
    // check image if it exists
    if (img_in == NULL) {
        printf("Error in loading the image.\n");
        printf("Please check if the image exists.\n");
        return 1; // exit the program with 1 error
    }
    // Debugging purposes: print image width, height, and channels
    printf("Image width: %d\n", width);
    printf("Image height: %d\n", height);
    printf("Number of channels: %d\n", channels);

    // Debugging purposes: to check if the image is in grayscale
    if (!stbi_write_jpg("./grayscale/input.jpg", width, height, GRAY, img_in, 100)) {
        printf("Error in saving the output image.\n");
        return 1; // exit the program with 1 error
    }


    // Generate Gaussian Blur mask
    generateGaussianBlurMask(gaussianMask);

    // Allocate memory for the output images
    unsigned char *img_out[NUM_DIRECTIONS];
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        img_out[i] = (unsigned char *)malloc(width * height * sizeof(unsigned char));
        if (img_out[i] == NULL) {
            printf("Error in allocating memory for the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Apply Gaussian Blur mask to input image before applying the RCM
    gaussianBlurMask(img_in, width, height, gaussianMask);
    const char filename_suffix[8][3] = {
        "N\0", "NW", "W\0", "SW", "S\0", "SE", "E\0", "NE"
    };
    // Apply the Robinson's Compass mask for each direction using CUDA
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        // Apply RCM
        robCompMaskCUDA(img_out[i], img_in, width, height, &ROBINSON_COMPASS_MASK[i][0][0]);
        // Generate output filename
        char direction[MAX_CHAR];
        snprintf(direction, MAX_CHAR, "cuda_%c%c%c", filename_suffix[i][0], filename_suffix[i][1], filename_suffix[i][2]);
        generateOutputFilename(filename, output_filename, direction);
        // Debugging purposes: print output filename
        printf("Output filename: %s\n", output_filename);
        // Save output image
        if (!stbi_write_jpg(output_filename, width, height, GRAY, img_out[i], 100)) {
            printf("Error in saving the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Free memory for the output images
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        free(img_out[i]);
    }

    // Free memory for the input image
    stbi_image_free(img_in);

    return 0;
}

Writing robinsonCompGBlur_CUDA.cu


In [ ]:
%%shell
nvcc robinsonCompGBlur_CUDA.cu -o robinsonCompGBlur_CUDA

stb_image.h(4262): warning #550-D: variable "old_limit" was set but never used

stb_image.h(5184): warning #550-D: variable "idata_limit_old" was set but never used

stb_image.h(6971): warning #550-D: variable "out_size" was set but never used

stb_image.h(6972): warning #550-D: variable "delays_size" was set but never used



In [ ]:
%%shell
nvprof ./robinsonCompGBlur_CUDA

Enter the image filename: cat_small.jpg
Filename: cat_small.jpg
Input filename: ./inputs/cat_small.jpg
Image width: 224
Image height: 225
Number of channels: 3
==2674== NVPROF is profiling process 2674, command: ./robinsonCompGBlur_CUDA
Output filename: ./outputs/cat_small_cuda_N.jpg
Output filename: ./outputs/cat_small_cuda_NW.jpg
Output filename: ./outputs/cat_small_cuda_W.jpg
Output filename: ./outputs/cat_small_cuda_SW.jpg
Output filename: ./outputs/cat_small_cuda_S.jpg
Output filename: ./outputs/cat_small_cuda_SE.jpg
Output filename: ./outputs/cat_small_cuda_E.jpg
Output filename: ./outputs/cat_small_cuda_NE.jpg
==2674== Profiling application: ./robinsonCompGBlur_CUDA
==2674== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.37%  591.44us         1  591.44us  591.44us  591.44us  gaussianBlurMaskKernel(unsigned char*, unsigned char*, int, int, double*)
                   10.69%  86.142us         8  10.767us  1

## Robinson Compass Mask CUDA with Gaussian Blur (with Grid-stride loop)

In [ ]:
%%writefile robinsonCompGBlur_CUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

// Headers for image input and output
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#define CUDA_CHECK(call)                                            \
do {                                                                \
    cudaError_t cudaStatus = call;                                  \
    if (cudaStatus != cudaSuccess) {                                \
        printf("CUDA Error at line %d: %s\n", __LINE__, cudaGetErrorString(cudaStatus)); \
        exit(1);                                                    \
    }                                                               \
} while (0)


#define MAX_CHAR 32
#define GRAY 1
#define RGB 3
#define MASK_SIZE 3
#define GAUSSIAN_MASK_SIZE 5
#define GAUSSIAN_SIGMA 5.0

// For the PI value in C
#ifndef M_PI
#   define M_PI 3.14159265358979323846
#endif



// =================================RCM KERNEL====================================================

// CUDA kernel to apply the Robinson Compass Mask
__global__ void robCompMaskKernel(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < width * height; i += stride) {
        int h = i / width;
        int w = i % width;

        if (h < height && w < width) {
            int sum = 0;
            for (int r = -1; r <= 1; r++) {
                for (int c = -1; c <= 1; c++) {
                    int idx_h = min(max(h + r, 0), height - 1);
                    int idx_w = min(max(w + c, 0), width - 1);
                    sum += img_in[idx_h * width + idx_w] * mask[(r + 1) * 3 + (c + 1)];
                }
            }
            img_out[h * width + w] = (unsigned char)abs(sum);
        }
    }
}


// Function to apply the Robinson Compass Mask using CUDA
void robCompMaskCUDA(unsigned char *img_out, unsigned char *img_in, int width, int height, const int *mask) {
    unsigned char *d_img_in, *d_img_out;
    int *d_mask;

    // Allocate device memory for input image, output image, and mask
    CUDA_CHECK(cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char)));
    CUDA_CHECK(cudaMalloc((void **)&d_mask, 9 * sizeof(int)));

    // Copy data from host to device
    CUDA_CHECK(cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_mask, mask, 9 * sizeof(int), cudaMemcpyHostToDevice));
    // Define block size and grid size
        //start here
    int numThreads = 1024;
    int numBlocks = ((width*height)+numThreads-1) / numThreads;
    //dim3 blockSize(32, 32); //same as 1024 threads per block
    //dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Launch the kernel
    robCompMaskKernel<<<numBlocks,  numThreads>>>(d_img_out, d_img_in, width, height, d_mask);

    // Copy result back to host
    CUDA_CHECK(cudaMemcpy(img_out, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Free device memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}


// =================================GAUSSIAN BLUR (FOR SMOOTHENING)====================================================
void generateGaussianBlurMask(double mask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE]) {
    // Declare the Gaussian mask
    double sum = 0;
    int max_index = GAUSSIAN_MASK_SIZE / 2;
    for (int r = -GAUSSIAN_MASK_SIZE/2; r <= max_index; r++) {
        for (int c = -GAUSSIAN_MASK_SIZE/2; c <= max_index; c++) {
            mask[r + max_index][c + max_index] = exp(-((double)(r * r + c * c)) / (double)(2 * GAUSSIAN_SIGMA * GAUSSIAN_SIGMA)) / (double)(2 * M_PI * GAUSSIAN_SIGMA * GAUSSIAN_SIGMA);
            sum += mask[r + max_index][c + max_index];
        }
    }

    // Normalize the Gaussian mask
    for (int r = 0; r < GAUSSIAN_MASK_SIZE; r++) {
        for (int c = 0; c < GAUSSIAN_MASK_SIZE; c++) {
            mask[r][c] /= sum;
        }
    }
}

__global__ void gaussianBlurMaskKernel(unsigned char *img_in, unsigned char *img_out, int width, int height, double *mask) {
    const int maxIndex = GAUSSIAN_MASK_SIZE / 2;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < width) {
        int sum = 0;
        int rowStart, colStart, maskRowIndex, maskColIndex, maxRow, maxCol;

        // Reset mask indices
        rowStart = -maxIndex;
        colStart = -maxIndex;
        maskRowIndex = 0;
        maskColIndex = 0;
        maxRow = GAUSSIAN_MASK_SIZE;
        maxCol = GAUSSIAN_MASK_SIZE;

        // Center mask to the current pixel
        if (row == 0) {
            rowStart = 0;
            maskRowIndex = maxIndex;
            maxRow = GAUSSIAN_MASK_SIZE;
        }
        if (row > 0 && row < maxIndex) {
            rowStart = -row;
            maskRowIndex = row;
            maxRow = GAUSSIAN_MASK_SIZE;
        }
        if (row >= height - maxIndex && row < height - 1) {
            rowStart = -maxIndex;
            maskRowIndex = 0;
            maxRow = GAUSSIAN_MASK_SIZE - (height - row - 1);
        }
        if (row == height - 1) {
            rowStart = -maxIndex;
            maskRowIndex = 0;
            maxRow = maxIndex + 1;
        }
        if (col == 0) {
            colStart = 0;
            maskColIndex = maxIndex;
            maxCol = GAUSSIAN_MASK_SIZE;
        }
        if (col > 0 && col < maxIndex) {
            colStart = -col;
            maskColIndex = col;
            maxCol = GAUSSIAN_MASK_SIZE;
        }
        if (col >= width - maxIndex && col < width - 1) {
            colStart = -maxIndex;
            maskColIndex = 0;
            maxCol = GAUSSIAN_MASK_SIZE - (width - col - 1);
        }
        if (col == width - 1) {
            colStart = -maxIndex;
            maskColIndex = 0;
            maxCol = maxIndex + 1;
        }

        // Apply the mask
        for (int r = rowStart, mr = maskRowIndex; mr < maxRow; r++, mr++) {
            for (int c = colStart, mc = maskColIndex; mc < maxCol; c++, mc++) {
                sum += img_in[(row + r) * width + (col + c)] * mask[mr * GAUSSIAN_MASK_SIZE + mc];
            }
        }

        // Set the output pixel to the sum
        img_out[row * width + col] = (unsigned char)abs(sum);
    }
}

void gaussianBlurMask(unsigned char *img_in, int width, int height, double mask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE]) {
    const int TILE_SIZE = 16;

    unsigned char *d_img_in, *d_img_out;
    double *d_mask;

    // Allocate memory on the GPU
    cudaMalloc((void **)&d_img_in, width * height * sizeof(unsigned char));
    cudaMalloc((void **)&d_img_out, width * height * sizeof(unsigned char));
    cudaMalloc((void **)&d_mask, GAUSSIAN_MASK_SIZE * GAUSSIAN_MASK_SIZE * sizeof(double));

    // Copy data from host to device
    cudaMemcpy(d_img_in, img_in, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, mask, GAUSSIAN_MASK_SIZE * GAUSSIAN_MASK_SIZE * sizeof(double), cudaMemcpyHostToDevice);

    dim3 threadsPerBlock(TILE_SIZE, TILE_SIZE);
    dim3 numBlocks((width + TILE_SIZE - 1) / TILE_SIZE, (height + TILE_SIZE - 1) / TILE_SIZE);

    gaussianBlurMaskKernel<<<numBlocks, threadsPerBlock>>>(d_img_in, d_img_out, width, height, d_mask);
    cudaDeviceSynchronize();

    // Copy the result back to the host
    cudaMemcpy(img_in, d_img_out, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost);

    // Free GPU memory
    cudaFree(d_img_in);
    cudaFree(d_img_out);
    cudaFree(d_mask);
}

// =================================INPUT AND OUTPUT FILE VALIDATOR & GENERATOR=================================

void generateOutputFilename(char* filename, char* output_filename, const char* direction) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != '.'; i++) {
        output_filename[i + 10] = filename[i];
    }
    output_filename[i + 10] = '_';
    i++;
    // copy the direction to the output filename
    int j;
    for (j = 0; direction[j] != 0; j++) {
        output_filename[i + 10 + j] = direction[j];
    }
    // add the filetype to the output filename
    output_filename[i + 10 + j] = '.';
    output_filename[i + 10 + j + 1] = 'j';
    output_filename[i + 10 + j + 2] = 'p';
    output_filename[i + 10 + j + 3] = 'g';
    output_filename[i + 10 + j + 4] = 0;
}

void generateInputFilename(char* filename, char* input_filename) {
    // copy the filename to the output filename
    int i;
    for (i = 0; i < MAX_CHAR && filename[i] != 0; i++) {
        input_filename[i + 9] = filename[i];
    }
}

int isFileTypeSupported(char* filename) {
    // Declare supported filetypes
    const char* supported_filetypes[] = {
        "jpg", "jpeg", "png", "bmp"
    };
    // Move to the period before the filetype
    int start = 0;
    for (int i = 0; i < MAX_CHAR && filename[start] != '.'; i++, start++);
    start++; // move to the first character of the filetype

    // Check if the filetype is supported
    for (int i = start, j = 0; j < 4 && filename[i] != 0; i++, j++) {
        // Debugging purposes: print the filename extension and the supported filetypes
        /* printf("%c - %c - %c - %c - %c\n",
            filename[i],
            supported_filetypes[0][j % 3],
            supported_filetypes[1][j % 4],
            supported_filetypes[2][j % 3],
            supported_filetypes[3][j % 3]); */

        // Check if the character is not equal to any of the supported file types
        if (filename[i] != supported_filetypes[0][j % 3]
            && filename[i] != supported_filetypes[1][j % 4]
            && filename[i] != supported_filetypes[2][j % 3]
            && filename[i] != supported_filetypes[3][j % 3]) {
            return 0; // return 0 if the filetype is not supported
        }
    }
    return 1; // return 1 if the filetype is supported
}

int main() {
    const int NUM_DIRECTIONS = 8;

    // Declare Robinson's Compass mask
      const int ROBINSON_COMPASS_MASK[8][3][3] = {
        {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}}, // N
        {{0, 1, 2}, {-1, 0, 1}, {-2, -1, 0}}, // NW
        {{1, 2, 1}, {0, 0, 0}, {-1, -2, -1}}, // W
        {{2, 1, 0}, {1, 0, -1}, {0, -1, -2}}, // SW
        {{1, 0, -1}, {2, 0, -2}, {1, 0, -1}}, // S
        {{0, -1, -2}, {1, 0, -1}, {2, 1, 0}}, // SE
        {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}}, // E
        {{-2, -1, 0}, {-1, 0, 1}, {0, 1, 2}}  // NE
    };

    // Declare Gaussian Blur Mask initialized to zero
    double gaussianMask[GAUSSIAN_MASK_SIZE][GAUSSIAN_MASK_SIZE] = { { 0 } };

    int width, height, channels;
    char filename[MAX_CHAR] = { 0 };
    char input_filename[MAX_CHAR + 16] = "./inputs/\0";
    char output_filename[MAX_CHAR + 16] = "./outputs/\0";
    unsigned char *img_in;

    // Ask for the image filename
    printf("Enter the image filename: ");
    scanf("%[^\n]%*c", filename);
    // Debugging purposes: print filename
    printf("Filename: %s\n", filename); // expected to print only one newline char

    // Check if filetype is supported for this program
    if (!isFileTypeSupported(filename)) {
        printf("File type not supported.\n");
        printf("Image should either be in JPG, PNG or BMP format.\n");
        return 1; // exit the program with 1 error
    }

    // Generate the input filename
    generateInputFilename(filename, input_filename);
    // Debugging purposes: print the input filename
    printf("Input filename: %s\n", input_filename);

    // Read the image
    img_in = stbi_load(input_filename, &width, &height, &channels, GRAY); // 4th param: GRAY or RGB
    // check image if it exists
    if (img_in == NULL) {
        printf("Error in loading the image.\n");
        printf("Please check if the image exists.\n");
        return 1; // exit the program with 1 error
    }
    // Debugging purposes: print image width, height, and channels
    printf("Image width: %d\n", width);
    printf("Image height: %d\n", height);
    printf("Number of channels: %d\n", channels);

    // Debugging purposes: to check if the image is in grayscale
    if (!stbi_write_jpg("./grayscale/input.jpg", width, height, GRAY, img_in, 100)) {
        printf("Error in saving the output image.\n");
        return 1; // exit the program with 1 error
    }


    // Generate Gaussian Blur mask
    generateGaussianBlurMask(gaussianMask);

    // Allocate memory for the output images
    unsigned char *img_out[NUM_DIRECTIONS];
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        img_out[i] = (unsigned char *)malloc(width * height * sizeof(unsigned char));
        if (img_out[i] == NULL) {
            printf("Error in allocating memory for the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Apply the Robinson's Compass mask for each direction using CUDA
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        // Apply Gaussian Blur mask to input image before applying the RCM
        gaussianBlurMask(img_in, width, height, gaussianMask);
        // Apply RCM
        robCompMaskCUDA(img_out[i], img_in, width, height, &ROBINSON_COMPASS_MASK[i][0][0]);
        // Generate output filename
        char direction[MAX_CHAR];
        snprintf(direction, MAX_CHAR, "Direction_%d", i);
        generateOutputFilename(filename, output_filename, direction);
        // Debugging purposes: print output filename
        printf("Output filename: %s\n", output_filename);
        // Save output image
        if (!stbi_write_jpg(output_filename, width, height, GRAY, img_out[i], 100)) {
            printf("Error in saving the output image.\n");
            return 1; // exit the program with 1 error
        }
    }

    // Free memory for the output images
    for (int i = 0; i < NUM_DIRECTIONS; i++) {
        free(img_out[i]);
    }

    // Free memory for the input image
    stbi_image_free(img_in);

    return 0;
}

In [ ]:
%%shell
nvcc robinsonCompGBlur_CUDA.cu -o robinsonCompGBlur_CUDA

In [ ]:
%%shell
nvprof ./robinsonCompGBlur_CUDA